# News recommendation serving
We will now implement a simple backend for our news serving example.

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import atoma
from flask import Flask, jsonify, request
from flask_cors import CORS
import requests
import ray

In [3]:
ray.init(num_cpus=2)

2019-04-09 18:30:06,399	INFO node.py:439 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-09_18-30-06_92151/logs.
2019-04-09 18:30:06,513	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:62990 to respond...
2019-04-09 18:30:06,635	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:26775 to respond...
2019-04-09 18:30:06,641	INFO services.py:761 -- Starting Redis shard with 1.72 GB max memory.
2019-04-09 18:30:06,670	INFO services.py:1449 -- Starting the Plasma object store with 2.58 GB memory using /tmp.


{'node_ip_address': None,
 'redis_address': '128.32.44.164:62990',
 'object_store_address': '/tmp/ray/session_2019-04-09_18-30-06_92151/sockets/plasma_store',
 'webui_url': None,
 'raylet_socket_name': '/tmp/ray/session_2019-04-09_18-30-06_92151/sockets/raylet'}

In [4]:
@ray.remote
class NewsServer(object):

    def __init__(self):
        pass

    def retrieve_feed(self, url):
        response = requests.get(url)
        feed = atoma.parse_rss_bytes(response.content)
        items = []
        for item in feed.items:
            items.append({"title": item.title,
                          "link": item.link,
                          "description": item.description,
                          "description_text": item.description,
                          "pubDate": str(item.pub_date)})

        return {"channel": {"title": feed.title,
                            "link": feed.link,
                            "url": feed.link},
                "items": items}

    def like_item(self, url, is_faved):
        if is_faved:
            print("url {} has been favorited".format(url))
        else:
            print("url {} has been defavorited".format(url))

In [ ]:
app = Flask("newsreader")
CORS(app)

@app.route("/api", methods=["POST"])
def dispatcher():
    req = request.get_json()
    method_name = req["method_name"]
    method_args = req["method_args"]
    if hasattr(dispatcher.server, method_name):
        method = getattr(dispatcher.server, method_name)
        # Doing a blocking ray.get right after submitting the task
        # might be bad for performance if the task is expensive.
        result = ray.get(method.remote(*method_args))
        return jsonify(result)
    else:
        return jsonify(
            {"error": "method_name '" + method_name + "' not found"})

dispatcher.server = NewsServer.remote()
app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
